# Clase 4 — Taller práctico: Desarrollo de una aplicación web interactiva para cargar y clasificar imágenes

---

## Objetivos de aprendizaje

Al finalizar la sesión los estudiantes deberán ser capaces de:

1. Diseñar la arquitectura de una aplicación web para visión por computador (frontend, API, modelo).
2. Implementar una API REST segura y eficiente que sirva un modelo de clasificación.
3. Implementar un cliente web interactivo (HTML/JS o React) que suba imágenes y muestre resultados.
4. Contenerizar la aplicación con Docker y ejecutar el sistema localmente o en la nube.
5. Aplicar buenas prácticas de ingeniería: PEP 8, tipado, logging, pruebas automatizadas y consideraciones de privacidad/seguridad (OWASP).

---


## Requisitos previos y recursos

* Python 3.9+
* Node.js + npm (para la parte de React opcional)
* Docker (para contenerización)
* Conocimiento básico de HTTP, REST y JSON

Paquetes principales (backend): `fastapi`, `uvicorn`, `pydantic`, `opencv-python` (si es necesario), `torch`/`torchvision` (si se usa modelo PyTorch), `python-multipart`.

---

## Arquitectura propuesta (alta prioridad)

1. **Frontend (cliente web)**

   * Página que permite seleccionar o arrastrar la imagen, muestra preview y permite enviar al backend.
   * Consume endpoints REST: `/health`, `/predict/upload`, `/predict/base64`.
2. **Backend (API)**

   * FastAPI con endpoints de inferencia, validación y logging.
   * Carga del modelo en `startup` (singleton) para evitar recargas por petición.
   * Manejo de límites de tamaño y validación MIME.
3. **Modelo**

   * Modelo de clasificación (ResNet18 u otro) cargado en memoria; para producción usar TorchScript/ONNX.
4. **Infraestructura**

   * Contenedores: backend y frontend (opcional). Docker Compose para orquestación local.
   * Observabilidad: logs estructurados y métricas básicas.

---

## Buenas prácticas, estándares y seguridad

* **PEP 8 / Pydantic / Tipado**: todo código Python debe tener docstrings, tipado y pasar flake8/black.
* **OWASP**: limitar tamaño de subida, validar tipo de archivo, evitar ejecución de datos inseguros, proteger endpoints con autenticación si necesario.
* **Privacidad**: no almacenar imágenes sin consentimiento; si se almacenan, cifrar y tener política de retención.
* **Testing**: `pytest` para unidad y `fastapi.testclient` para integración.
* **CI/CD**: usar GitHub Actions para ejecutar tests y linters en cada PR.

---



## Parte práctica: 

A continuación, se presentan instrucciones detalladas, código de ejemplo y ejercicios. El ejemplo usa **FastAPI** en el backend y **vanilla JS** para el frontend (luego se presenta una versión React opcional).

### 1) Estructura del proyecto

```
cv-webapp/
├── backend/
│   ├── app/
│   │   ├── main.py
│   │   ├── model.py
│   │   ├── schemas.py
│   │   └── utils.py
│   ├── requirements.txt
│   └── Dockerfile
├── frontend/
│   ├── index.html        # cliente HTML/JS simple
│   ├── app.js
│   └── Dockerfile (opcional)
├── docker-compose.yml
└── README.md
```

### 2) Backend: `app/main.py` (esqueleto)

> Nota: no incluyas en producción `debug` ni `--reload`.



In [ ]:
# backend/app/main.py
from fastapi import FastAPI, UploadFile, File, HTTPException, Depends
from fastapi.middleware.cors import CORSMiddleware
from starlette.requests import Request
import logging

from .model import VisionModel
from .schemas import PredictResponse

logger = logging.getLogger("cv_backend")
app = FastAPI(title="CV WebApp API")

# CORS: en clase permitir localhost; en producción restringir orígenes
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000", "http://127.0.0.1:3000"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

model: VisionModel | None = None

@app.on_event("startup")
def startup_event():
    global model
    model = VisionModel(device="cpu")  # o "cuda" si GPU disponible
    logger.info("Modelo cargado en startup")

@app.get("/health")
def health():
    return {"status": "ok"}

@app.post("/predict/upload", response_model=PredictResponse)
async def predict_upload(file: UploadFile = File(...)):
    if not file.content_type.startswith("image/"):
        raise HTTPException(status_code=400, detail="Archivo no es imagen")
    contents = await file.read()
    if len(contents) == 0:
        raise HTTPException(status_code=400, detail="Archivo vacío")
    # límite de tamaño (ejemplo 5MB)
    if len(contents) > 5 * 1024 * 1024:
        raise HTTPException(status_code=413, detail="Archivo demasiado grande")
    result = model.predict_bytes(contents)
    return result


### 3) Backend: `app/model.py` (carga del modelo y preprocesado)

Incluye: carga en `__init__`, preprocesado con `torchvision.transforms`, y funciones: `predict_bytes`, `predict_image`.



In [ ]:
# backend/app/model.py
from PIL import Image
import io
import torch
import torchvision.transforms as T
from torchvision import models

class VisionModel:
    def __init__(self, device: str = "cpu") -> None:
        self.device = device
        self.model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.model.eval().to(self.device)
        self.transform = T.Compose([
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
        ])

    def preprocess(self, image: Image.Image) -> torch.Tensor:
        return self.transform(image).unsqueeze(0).to(self.device)

    def predict(self, image: Image.Image) -> dict:
        x = self.preprocess(image)
        with torch.no_grad():
            logits = self.model(x)
            probs = torch.nn.functional.softmax(logits, dim=1)
            top_prob, top_cls = torch.max(probs, 1)
        return {"class_id": int(top_cls[0].item()), "score": float(top_prob[0].item())}

    def predict_bytes(self, data: bytes) -> dict:
        image = Image.open(io.BytesIO(data)).convert("RGB")
        return self.predict(image)


### 4) Backend: Pydantic schemas `app/schemas.py`



In [ ]:
from pydantic import BaseModel

class PredictResponse(BaseModel):
    class_id: int
    score: float


### 5) Frontend simple (HTML + JS)

Archivo `frontend/index.html`:



In [ ]:
<!doctype html>
<html>
<head>
  <meta charset="utf-8" />
  <title>CV WebApp - Demo</title>
</head>
<body>
  <h1>Sube una imagen</h1>
  <input id="fileInput" type="file" accept="image/*" />
  <button id="sendBtn">Enviar</button>
  <div id="preview"></div>
  <div id="result"></div>

  <script src="/app.js"></script>
</body>
</html>


Archivo `frontend/app.js` (cliente minimal):



In [ ]:
const fileInput = document.getElementById('fileInput');
const sendBtn = document.getElementById('sendBtn');
const preview = document.getElementById('preview');
const result = document.getElementById('result');

fileInput.addEventListener('change', () => {
  const file = fileInput.files[0];
  if (!file) return;
  const img = document.createElement('img');
  img.src = URL.createObjectURL(file);
  img.style.maxWidth = '300px';
  preview.innerHTML = '';
  preview.appendChild(img);
});

sendBtn.addEventListener('click', async () => {
  const file = fileInput.files[0];
  if (!file) { alert('Selecciona una imagen'); return; }
  const fd = new FormData();
  fd.append('file', file);

  const resp = await fetch('http://localhost:8000/predict/upload', {
    method: 'POST', body: fd
  });
  const json = await resp.json();
  result.innerText = JSON.stringify(json);
});


> En la clase explique CORS y por qué el `fetch` falla si no se configura correctamente en `FastAPI`.

### 6) Versión React (opcional)

Proveer un `create-react-app` con un componente `ImageUploader.jsx` que haga lo mismo (mostrar preview, barra de progreso, resultado). Incluir manejo de estado, accesibilidad (aria-labels) y pruebas con React Testing Library.

### 7) Contenerización y orquestación

* `backend/Dockerfile` (liviana):

```dockerfile
FROM python:3.10-slim
WORKDIR /app
COPY requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt
COPY app ./app
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"]
```

* `frontend/Dockerfile` (si sirve archivos estáticos con `http-server` o sirve desde Nginx)

* `docker-compose.yml` (ejemplo):

```yaml
version: '3.8'
services:
  backend:
    build: ./backend
    ports: ['8000:8000']
  frontend:
    build: ./frontend
    ports: ['3000:80']
```

Explicar cómo probar localmente con `docker compose up --build`.

### 8) Exportar modelo a TorchScript para producción

* Script `scripts/export_torchscript.py` traza `resnet18` y guarda `models/resnet18_traced.pt`.
* En `VisionModel` primero intenta cargar `resnet18_traced.pt` con `torch.jit.load` y, si no existe, carga modelo Python.

**Ventajas**: portabilidad y a veces mejoras de latencia en CPU. Explicar limitaciones (por ejemplo detección, operaciones dinámicas).



## Actividades y ejercicios

1. Implementar la aplicación completa y correrla localmente.
2. Añadir endpoint `/predict/base64` y escribir cliente JavaScript para enviar imágenes como base64.
3. Exportar y cargar el modelo TorchScript; medir latencias antes/después.
4. Añadir autenticación básica (API Key) y documentar el proceso.
5. Escribir tests y configurar GitHub Actions que ejecuten `pytest` y `flake8` en cada PR.

---



## Recursos y lecturas recomendadas

* FastAPI docs (endpoints, CORS, UploadFile)
* PyTorch/TorchVision docs (model zoo, transforms, TorchScript)
* OWASP (mitigación de vulnerabilidades web, file upload)
* Artículos sobre deployment de modelos (TorchServe, ONNX Runtime, TensorFlow Serving)

---

